In [112]:
import torch
from torch import Tensor
from torch.autograd import Variable
from torch import nn
from torch.nn import functional as F

from helpers import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import dlc_bci as bci
train_input , train_target = bci.load(root = './data_bci')
print(str(type(train_input)), train_input.size()) 
print(str(type(train_target)), train_target.size())
test_input , test_target = bci.load(root = './data_bci', train = False)
print(str(type(test_input)), test_input.size()) 
print(str(type(test_target)), test_target.size())

<class 'torch.FloatTensor'> torch.Size([316, 28, 50])
<class 'torch.LongTensor'> torch.Size([316])
<class 'torch.FloatTensor'> torch.Size([100, 28, 50])
<class 'torch.LongTensor'> torch.Size([100])


In [2]:
import dlc_bci as bci
train_input , train_target = bci.load(root = './data_bci', one_khz = True)
print(str(type(train_input)), train_input.size()) 
print(str(type(train_target)), train_target.size())
test_input , test_target = bci.load(root = './data_bci', train = False, one_khz = True)
print(str(type(test_input)), test_input.size()) 
print(str(type(test_target)), test_target.size())

<class 'torch.FloatTensor'> torch.Size([316, 28, 500])
<class 'torch.LongTensor'> torch.Size([316])
<class 'torch.FloatTensor'> torch.Size([100, 28, 500])
<class 'torch.LongTensor'> torch.Size([100])


In [3]:
train_input[0]


  41.8000   44.8000   47.1000  ...    69.8000   72.6000   76.1000
 -10.3000   -5.9000   -3.3000  ...    12.6000   24.0000   26.5000
  38.1000   25.2000   46.0000  ...    45.1000   74.1000   64.8000
             ...                ⋱                ...             
   7.9000   11.2000   14.3000  ...    32.7000   43.4000   45.5000
  19.2000   33.6000   33.8000  ...    46.7000   53.7000   43.4000
  -0.4000   12.7000   12.0000  ...    30.7000   40.6000   33.1000
[torch.FloatTensor of size 28x50]

In [4]:
train_target


 0
 1
 0
 1
 0
 1
 0
 1
 0
 0
 1
 1
 0
 0
 1
 1
 1
 0
 0
 1
 0
 0
 1
 0
 1
 0
 0
 0
 0
 0
 0
 0
 1
 1
 1
 1
 1
 1
 0
 1
 0
 1
 0
 0
 1
 0
 1
 1
 0
 1
 1
 0
 0
 1
 0
 0
 1
 0
 1
 0
 1
 0
 0
 1
 0
 0
 0
 0
 0
 1
 0
 1
 0
 0
 0
 0
 0
 1
 1
 1
 0
 1
 0
 1
 1
 1
 1
 1
 1
 1
 1
 1
 0
 0
 0
 0
 1
 1
 1
 0
 1
 1
 1
 0
 1
 0
 1
 1
 1
 0
 1
 0
 1
 0
 0
 0
 1
 0
 0
 1
 1
 1
 1
 0
 1
 1
 0
 0
 1
 1
 1
 1
 1
 0
 0
 0
 0
 1
 1
 0
 1
 1
 1
 1
 1
 1
 1
 0
 0
 0
 1
 1
 0
 0
 1
 1
 0
 0
 0
 0
 0
 1
 1
 1
 1
 1
 1
 0
 1
 0
 1
 1
 0
 1
 0
 0
 1
 1
 1
 0
 0
 1
 0
 0
 0
 0
 0
 1
 1
 0
 0
 0
 0
 1
 0
 0
 0
 1
 0
 0
 1
 0
 0
 0
 1
 1
 1
 0
 0
 0
 1
 1
 1
 0
 0
 0
 1
 0
 1
 1
 1
 1
 1
 1
 1
 0
 1
 0
 0
 0
 1
 0
 1
 1
 1
 0
 1
 1
 0
 0
 0
 1
 1
 0
 1
 1
 0
 1
 1
 1
 1
 1
 0
 1
 0
 1
 0
 0
 0
 0
 0
 0
 1
 1
 1
 0
 1
 1
 1
 0
 1
 0
 0
 1
 1
 0
 0
 1
 0
 0
 1
 0
 0
 0
 1
 0
 0
 1
 1
 0
 0
 1
 1
 1
 0
 1
 0
 1
 1
 1
 0
 1
 1
 0
 1
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
[torch.LongTensor of size 316]

## Logistic Regression

In [5]:
from sklearn.linear_model import LogisticRegression

In [6]:
logistic_reg = LogisticRegression()

In [7]:
logistic_reg.fit(train_input.view(train_input.size(0),-1),train_target)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [8]:
pred = logistic_reg.predict(test_input.view(test_input.size(0),-1))

In [9]:
100*(1-(torch.LongTensor(pred) - test_target).abs().sum()/test_input.size(0))

74.0

## Neural Network

In [129]:
class Net(nn.Module):
    def __init__(self, nb_hidden=64):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.fc1 = nn.Linear(256, nb_hidden)
        self.fc2 = nn.Linear(nb_hidden, 2)

    def forward(self, x):
        print("======")
        print(x.size())
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=3, stride=3))
        print(x.size())
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size=2, stride=2))
        print(x.size())
        x = F.relu(self.fc1(x.view(-1, 256)))
        print(x.size())
        x = self.fc2(x)
        print(x.size())
        print("======")
        return x

In [130]:
my_cool_net = Net()

In [131]:
train_model(my_cool_net, Variable(train_input.view(-1, 1, 28, 50)), Variable(train_target))

torch.Size([316, 1, 28, 50])
torch.Size([316, 32, 8, 15])
torch.Size([316, 64, 2, 5])
torch.Size([790, 64])
torch.Size([790, 2])
torch.Size([790, 2])
torch.Size([316])


RuntimeError: Assertion `THIndexTensor_(size)(target, 0) == batch_size' failed.  at /Users/soumith/code/builder/wheel/pytorch-src/torch/lib/THNN/generic/ClassNLLCriterion.c:79

In [115]:
nb_errors = compute_nb_errors(net, Variable(test_input.view(test_input.size(0),-1)), Variable(test_target))

In [101]:
nb_errors/test_input.size(0)

0.51

In [102]:
for p in net.parameters():
    print(p)

Parameter containing:
        inf         nan         nan  ...          nan         nan         nan
        inf         nan         nan  ...          nan         nan         nan
        inf         nan         nan  ...          nan         nan         nan
                ...                   ⋱                   ...                
        inf         nan         nan  ...          nan         nan         nan
        inf         nan         nan  ...          nan         nan         nan
        inf         nan         nan  ...          nan         nan         nan
[torch.FloatTensor of size 2000x1400]

Parameter containing:
1.00000e-02 *
     inf
     inf
     inf
    ⋮   
     inf
     inf
     inf
[torch.FloatTensor of size 2000]

Parameter containing:
        inf         inf         inf  ...          inf         inf         inf
 7.4822e-04  1.8017e-02  4.9546e-03  ...  -1.0733e-02 -1.6233e-02 -2.1673e-03
 9.0141e-03  1.6760e-02 -1.5477e-02  ...   1.0472e-02 -5.0546e-03 -7.6107e-04
    

In [95]:
torch.max(net(Variable(test_input.view(test_input.size(0),-1))),1)

(Variable containing:
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 nan
 [torch.FloatTensor of size 100], Variable containing:
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
 [torch.LongTensor of s

In [83]:
net(Variable(test_input.view(test_input.size(0),-1)))

Variable containing:
-2.6572  0.0148
-4.1458  1.6060
-3.4840  0.6600
-3.8114  0.2711
-2.0816 -0.4006
-1.6004  0.9984
-2.2228 -0.6758
-2.2082 -0.5414
-2.1721 -0.0312
-3.9432  0.9518
-3.7670  1.3886
-2.7920  1.0095
-1.8241  1.2546
-2.2338  0.1086
-2.8144 -0.1049
-1.6607  0.2411
-4.0314  0.7664
-2.6379  1.1434
-2.5038  1.0571
-2.2371 -0.2091
-2.4405  0.6255
-3.3993  0.3146
-2.8773  0.0630
-7.2798  2.1457
-2.5702  1.2132
-2.8056  0.0115
-3.5939  1.7304
-3.0597  0.0963
-3.4168  0.0629
-2.3354  0.3484
-2.8556  0.5807
-3.0493  0.7753
-2.7777  0.6547
-2.1945 -0.6982
-3.2607  0.5900
-2.9778  0.8725
-2.3618  0.0959
-2.4101 -0.3257
-2.9874  1.2907
-3.5630 -0.1187
-2.9051  1.1212
-3.4610  0.5801
-2.8010  0.4318
-2.3829  0.2838
-2.5511  0.1325
-2.0369 -0.3338
-3.3242  0.5825
-2.2085 -0.6308
-4.3520  2.9381
-5.0417  2.9876
-5.5641  1.3397
-2.8681  0.2895
-2.7957  1.0283
-2.3342 -0.3569
-1.6148 -0.5689
-4.3562 -0.1665
-2.5958  0.8790
-3.4593  1.1641
-2.2065  2.0716
-3.5562  1.0820
-2.2726 -0.0875
-4.